In [ ]:
import xarray as xr 
from preprocessing.utils import scaleLongitudes
import geopandas as gpd
import pandas as pd

#convert a list of tuples into a pandas dataframe
def convertToDF(list_of_tuples):
    df = pd.DataFrame(list_of_tuples, columns = ['lat','lon']) 
    return df

def getEcoZoneCoordinates():
    ecozones = gpd.read_file('data/shapefiles/ecozones.shp').to_crs('epsg:4326')
    netcdf_file = xr.open_dataset(f'data/CESM/treeFrac_Lmon_CESM2_historical_r11i1p1f1_gn_199901-201412.nc')
    netcdf_file = scaleLongitudes(netcdf_file)
    #to make sure the center of the cell is being considered for the clipping
    netcdf_file['lat'] = netcdf_file['lat'] + 0.5
    netcdf_file['lon'] = netcdf_file['lon'] + 0.75
    dissolved_ecozones = ecozones.dissolve(by='ZONE_NAME').reset_index()
    netcdf_file = netcdf_file['treeFrac']
    df = pd.DataFrame()
    for _,region in dissolved_ecozones.iterrows():
        gdf = gpd.GeoDataFrame(geometry=[region.geometry])
        netcdf_file.rio.set_spatial_dims(x_dim="lon", y_dim="lat", inplace=True)
        netcdf_file.rio.write_crs("epsg:4326", inplace=True)
        clipped = netcdf_file.rio.clip(gdf.geometry, ecozones.crs,drop=True)
        clipped['lat'] = clipped['lat'] - 0.5
        clipped['lon'] = clipped['lon'] - 0.75
        stacked = clipped.stack(x=['lat','lon'])
        val = stacked.isel(time=0)[stacked.isel(time=0).notnull()].coords['x'].values
        x = convertToDF(list(val))
        x['zone'] = region['ZONE_NAME']
        df= pd.concat([df,x])
    df['lat'] = df['lat'].round(7)
    return df

In [ ]:
#SHOW STUDY AREA
import matplotlib.pyplot as plt
import geopandas 

ecozones = geopandas.read_file('data/shapefiles/ecozones.shp').to_crs('epsg:4326')
canada = geopandas.read_file('data/shapefiles/lpr_000b16a_e/lpr_000b16a_e.shp').to_crs('epsg:4326')
fig, ax = plt.subplots(figsize=(10,10))
canada.plot(ax=ax,color='white',edgecolor='black')
ax.legend(['Boreal Shield','Boreal Cordillera','Boreal Plain'],fontsize=30,loc='upper left')
ecozones.where(ecozones['ZONE_NAME'].isin(['Boreal Shield','Boreal Cordillera','Boreal PLain'])).plot(column='ZONE_NAME',ax=ax,cmap='cividis',legend=True,legend_kwds={'fontsize':15})
# ax.set_title('Study Area',fontsize=40)


In [85]:
observed_ds = pd.read_csv(f'{cfg.data}/observed_timeseries{cfg.model.seq_len}_data.csv')

nfis_data = pd.read_csv(f'{cfg.data}/forest_df.csv')
# era_data = era_data.where(era_data['stl1'] > 0).dropna()
# nfis_data = nfis_data.where(nfis_data['broadleaf'] > 0).dropna()
df_merged = pd.merge(observed_ds,nfis_data,on=['year','lat','lon'],how='left')

In [86]:
observed_ds

,ps,tsl,treeFrac,pr,tas_DJF,tas_JJA,year,lat,lon
0,85937.445312,274.260651,58.928241,0.000027,262.054047,281.602417,1985.0,56.073299,-126.25
1,85819.335938,274.391357,59.623726,0.000030,261.285065,281.731079,1986.0,56.073299,-126.25
2,85839.835938,274.904999,60.231902,0.000035,261.349701,282.326691,1987.0,56.073299,-126.25
3,85815.382812,274.764435,60.392790,0.000033,259.192383,281.280701,1988.0,56.073299,-126.25
4,85958.960938,275.752411,60.537298,0.000032,260.242920,284.144135,1989.0,56.073299,-126.25
...,...,...,...,...,...,...,...,...,...
66955,98410.601562,277.766815,27.779927,0.000015,256.222260,290.260590,2015.0,58.900524,-110.00
66956,98424.843750,278.532074,28.984158,0.000019,256.728729,290.546875,2016.0,58.900524,-110.00
66957,98406.187500,278.284760,30.621631,0.000012,257.128937,291.080475,2017.0,58.900524,-110.00
66958,98541.710938,277.154633,31.274102,0.000012,255.206680,290.209259,2018.0,58.900524,-110.00


In [87]:
df_merged['percent_harvested'] = df_merged['percent_harvested'] * 100
df_merged['percentage_growth'] = df_merged['percentage_growth'] * 100
df_merged['without_regrowth'] = df_merged['treeFrac'] - df_merged['percentage_growth']
df_merged['reforested'] = df_merged['treeFrac'] +  df_merged['percent_harvested']
df_merged.drop(columns=['treeFrac'],inplace=True)

reforestation = df_merged.rename(columns={'reforested':'treeFrac'})


In [88]:
reforestation

,ps,tsl,pr,tas_DJF,tas_JJA,year,lat,lon,percentage_growth,tree_cover,percent_harvested,without_regrowth,treeFrac
0,85937.445312,274.260651,0.000027,262.054047,281.602417,1985.0,56.073299,-126.25,0.000000,0.589282,0.072246,58.928241,59.000487
1,85819.335938,274.391357,0.000030,261.285065,281.731079,1986.0,56.073299,-126.25,0.000081,0.596237,0.050959,59.623645,59.674685
2,85839.835938,274.904999,0.000035,261.349701,282.326691,1987.0,56.073299,-126.25,0.000422,0.602319,0.091096,60.231480,60.322998
3,85815.382812,274.764435,0.000033,259.192383,281.280701,1988.0,56.073299,-126.25,0.000601,0.603928,0.168590,60.392189,60.561380
4,85958.960938,275.752411,0.000032,260.242920,284.144135,1989.0,56.073299,-126.25,0.000861,0.605373,0.086464,60.536437,60.623762
...,...,...,...,...,...,...,...,...,...,...,...,...,...
66955,98410.601562,277.766815,0.000015,256.222260,290.260590,2015.0,58.900524,-110.00,0.000179,0.277799,0.004581,27.779749,27.784508
66956,98424.843750,278.532074,0.000019,256.728729,290.546875,2016.0,58.900524,-110.00,0.000520,0.289842,0.000211,28.983638,28.984369
66957,98406.187500,278.284760,0.000012,257.128937,291.080475,2017.0,58.900524,-110.00,0.000341,0.306216,0.002583,30.621289,30.624213
66958,98541.710938,277.154633,0.000012,255.206680,290.209259,2018.0,58.900524,-110.00,0.000227,0.312741,0.000000,31.273875,31.274102


In [89]:
from infer_lstm import infer_lstm
reforested_carbon = infer_lstm(reforestation,cfg)

/Users/gclyne/thesis/env/lib/python3.9/site-packages/sklearn/base.py:288: UserWarning: Trying to unpickle estimator StandardScaler from version 1.1.2 when using version 1.2.0. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
/Users/gclyne/thesis/infer_lstm.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.loc[:,cfg.model.input ] =scaler.transform(df.loc[:,cfg.model.input])


In [135]:
emulated = pd.read_csv(f'{cfg.data}/emulation_df.csv')


In [136]:
emulated
#select all rows that contain NaN values
emulated[emulated.isna().any(axis=1)][['lat','lon']].drop_duplicates()

,lat,lon


In [137]:
input_data = pd.read_csv(f'{cfg.data}/observed_timeseries{cfg.model.seq_len}_data.csv')
input_data = input_data.groupby(['year','lat','lon']).mean().reset_index()
input_data[input_data.isna().any(axis=1)]

,year,lat,lon,ps,tsl,treeFrac,pr,tas_DJF,tas_JJA


In [ ]:
#drop all rows with nan
